In [7]:
# import pandas as pd
# answer_df = pd.read_csv("../../data2/answer_df_vicuna.csv")
# answer_df.iloc[0:500].to_csv("../../data2/answer_df_vicuna1.csv", index=False)
# answer_df.iloc[500:1000].to_csv("../../data2/answer_df_vicuna2.csv", index=False)
# answer_df.iloc[1000:].to_csv("../../data2/answer_df_vicuna3.csv", index=False)


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch


print(torch.cuda.is_available())
if torch.cuda.is_available():
    torch.cuda.empty_cache()
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available, otherwise fallback to CPU
model_id = "lmsys/vicuna-7b-v1.5"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             torch_dtype=torch.bfloat16, 
                                             device_map=device)



True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/jq2uw/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/jq2uw/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/jq2uw/.local/lib/python3.11/site-packages/transformers

In [8]:
piece_id = 1
answer_df_path = "../../data2/answer_df_vicuna"+str(piece_id)+".csv"

In [9]:
import pandas as pd
import numpy as np
answer_df = pd.read_csv(answer_df_path)
query_df = pd.read_csv("../../data2/fea_df.csv")

if not 'answer_string' in answer_df.columns:
    answer_df['answer_string'] = np.nan
    
print(answer_df.shape)
print(answer_df.iloc[0:10,])
query_df

(500, 3)
   sm_id                                         text_w_eos  answer_string
0   1690  i saw a cardiologist twice and was told i did ...            NaN
1    724       sharing my weight with me when asked not to.            NaN
2   2800  'i do not think it is an eating disorder.'   a...            NaN
3   1771  when i was severely underweight my pcp suggest...            NaN
4   1975                primary care providers not educated            NaN
5    126  the first time i sought a therapist they state...            NaN
6   1511  unhelpful more than damaging more likely in th...            NaN
7   4091  most of the 40 years i had a diagnosis of bn p...            NaN
8    371  cut out food groups to help with physical disa...            NaN
9   4313  primary care provider told me to 'go eat a che...            NaN


,fea,description
0,weightbasedhealth,Judgments about health based on weight or appe...
1,weightstigma,"Negative attitudes, discrimination, or prejudi..."
2,weightbasedhealthcare,Healthcare decisions influenced by weight. Not...
3,encourageweightloss,Encouragement of weight loss
4,dietpromotion,Advocacy for diets or restrictive eating
5,thinness,Reassurance of thinness. Maintaining thinness
6,notlowweightenough,"Not sick enough, or not thin enought, or not l..."
7,weightblame,Weight blamed for health issues or concerns
8,weightvalue,Weight tied to personality characteristics


In [10]:
# prompt = "Tell me about AI"
# prompt_template=f'''A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: {prompt} ASSISTANT:
# '''
# print("\n\n*** Generate:")

# input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
# print(tokenizer.decode(output[0]))


In [11]:
system_message = """
You are an AI assistant designed to answer questions.
Use the exact answer format asked.
"""

def format_user_message(text):
    question_content = "The provided parapraph is eating disorders patients' experiences with physicians. Weight stigma is indicated by weight-or-appearance-related treatment or judgement in this context. Does the paragraph mention any of the following topics:\n"
    for i in range(len(query_df)):
        question_content += f"  ({i+1}) {query_df.fea[i]}: {query_df.description[i]}.\n"
    answer_content = "Return answer in format: Answer:\n"
    for i in range(len(query_df)): 
        answer_content += f"  ({i+1}) {query_df.fea[i]}: yes/no, related phrases in the paragraph if any: \n"
    paragragh_content = f"Paragraph: '{text}' \n"
    user_message = system_message + question_content + paragragh_content + answer_content
    #print(user_message)
    
    return user_message




In [12]:
from tqdm import tqdm

for k in tqdm(range(1000), desc="Processing batch"):
    
    batch_size = 10

    # Filter for rows where 'answer_string' is NaN
    unanswered_df = answer_df[answer_df['answer_string'].isna()]

    # Get the indices of these NaN entries in the original DataFrame
    indices_to_update = unanswered_df.index[:batch_size]

    # Prepare prompt content for the first 10 entries with NaN answer_string
    user_messages = [format_user_message(text) for text in unanswered_df['text_w_eos'].iloc[:batch_size]]

    # Save the indices list if needed for later use
    indices_to_update_list = list(indices_to_update)

    if len(indices_to_update_list)>0:

        for i in range(len(user_messages)):
            failed = True
            while failed:
                input_ids = tokenizer(user_messages[i], return_tensors='pt').input_ids.to(model.device)
                output = model.generate(inputs=input_ids, 
                                        temperature=0.1,
                                        do_sample=True, 
                                        max_new_tokens=512)
                response = tokenizer.decode(output[0], skip_special_tokens=True)

                if "Answer:" in response:
                    answer = response.split("Answer:")[-1]
                    # Use the original index from indices_to_update_list
                    answer_df.loc[indices_to_update_list[i], 'answer_string'] = answer
                    # print(answer)
                    failed = False
                else:
                    failed = True
                    # Use the original index from indices_to_update_list
                    answer_df.loc[indices_to_update_list[i], 'answer_string'] = "Answer not found"

        print(answer_df.loc[indices_to_update_list, ['sm_id','answer_string']])
        answer_df.to_csv(answer_df_path, index=False)
        
    else:
        break
    

Processing batch:   0%|          | 1/1000 [01:25<23:43:22, 85.49s/it]

   sm_id                                      answer_string
0   1690  \n(1) weightbasedhealth: yes, related phrases ...
1    724  \n(1) weightbasedhealth: yes, related phrases ...
2   2800  \n(1) weightbasedhealth: No.\n(2) weightstigma...
3   1771  \n(1) weightbasedhealth: yes, related phrases ...
4   1975  \n(1) weightbasedhealth: yes, related phrases ...
5    126  \n(1) weightbasedhealth: yes, "weight or appea...
6   1511  \n(1) weightbasedhealth: yes, related phrases ...
7   4091  \n(1) weightbasedhealth: yes, related phrases ...
8    371  \n(1) weightbasedhealth: yes, related phrases ...
9   4313  \n(1) weightbasedhealth: yes, "weight or appea...


Processing batch:   0%|          | 2/1000 [03:01<25:28:27, 91.89s/it]

    sm_id                                      answer_string
10    430  \n(1) weightbasedhealth: yes, related phrases ...
11     70  \n(1) weightbasedhealth: yes, related phrases ...
12   2225  \n(1) weightbasedhealth: yes, related phrases ...
13    578  \n(1) weightbasedhealth: yes, related phrases ...
14    913  \n(1) weightbasedhealth: Yes, related phrases ...
15   2367  \n(1) weightbasedhealth: yes, related phrases ...
16   4577  \n(1) weightbasedhealth: yes, related phrases ...
17   2536  \n(1) weightbasedhealth: yes, related phrases ...
18   1085  \n(1) weightbasedhealth: yes, related phrases ...
19   3916  \n(1) weightbasedhealth: yes, related phrases ...


Processing batch:   0%|          | 3/1000 [04:24<24:14:27, 87.53s/it]

...

     sm_id                                      answer_string
470    506  \n(1) weightbasedhealth: yes, related phrases ...
471   2526  \n(1) weightbasedhealth: yes, related phrases ...
472    788  \n(1) weightbasedhealth: yes, related phrases ...
473    269  \n(1) weightbasedhealth: yes, related phrases ...
474   1410  \n(1) weightbasedhealth: yes, related phrases ...
475   4265  \n(1) weightbasedhealth: yes, related phrases ...
476   4365  \n(1) weightbasedhealth: yes, related phrases ...
477    998  \n(1) weightbasedhealth: yes, related phrases ...
478    605  \n(1) weightbasedhealth: yes, related phrases ...
479   2362  \n(1) weightbasedhealth: yes, related phrases ...


Processing batch:   5%|▍         | 49/1000 [1:13:38<25:20:55, 95.96s/it]

     sm_id                                      answer_string
480   2352  \n(1) weightbasedhealth: yes, related phrases ...
481   2744  \n(1) weightbasedhealth: yes, related phrases ...
482   2617  \n(1) weightbasedhealth: yes, related phrases ...
483   2448  \n(1) weightbasedhealth: yes, related phrases ...
484   1748  \n(1) weightbasedhealth: yes, "weight or appea...
485   3517  \n(1) weightbasedhealth: Yes, related phrases ...
486   1642  \n(1) weightbasedhealth: yes, related phrases ...
487   1769  \n(1) weightbasedhealth: yes, related phrases ...
488   3297  \n(1) weightbasedhealth: Yes, related phrases ...
489   1595  \n(1) weightbasedhealth: Yes, related phrases ...


Processing batch:   5%|▌         | 50/1000 [1:15:10<23:48:11, 90.20s/it]

     sm_id                                      answer_string
490   1892  \n(1) weightbasedhealth: yes, related phrases ...
491   1764  \n(1) weightbasedhealth: yes, related phrases ...
492   2053  \n(1) weightbasedhealth: yes, related phrases ...
493   2192  \n(1) weightbasedhealth: yes, related phrases ...
494   1650  \n(1) weightbasedhealth: yes, related phrases ...
495   4779  \n(1) weightbasedhealth: yes, related phrases ...
496    787  \n(1) weightbasedhealth: yes, related phrases ...
497   2563  \n(1) weightbasedhealth: yes, related phrases ...
498    508  \n(1) weightbasedhealth: yes, related phrases ...
499   3670  \n(1) weightbasedhealth: yes, related phrases ...


In [13]:
import pandas as pd

# File paths
file_paths = [
    "../../data2/answer_df_vicuna1.csv",
    "../../data2/answer_df_vicuna2.csv",
    "../../data2/answer_df_vicuna3.csv"
]

# Read and concatenate dataframes
answer_df_list = [pd.read_csv(file_path) for file_path in file_paths]
answer_df = pd.concat(answer_df_list, ignore_index=True)

# Save the concatenated dataframe
output_path = "../../data2/answer_df_vicuna.csv"
answer_df.to_csv(output_path, index=False)
